# CS 135 day02: Basics of Regression Methods

# Objectives

* Practice loading data from CSV files and manipulating that data with Pandas and NumPy
* Try out sklearn's linear regression on a dataset with 1-dim. features and many features
* Try out sklearn's k-nearest neighbor regression on a dataset with 1-dim. features and many features
* Try out comparing models using mean squared error as a performance metric

# What to do

Download the *abalone* dataset from our course starter-code repo here:

* Dataset CSV files: https://github.com/tufts-ml-courses/cs135-26s-assignments/tree/main/labs/data_abalone
* Dataset README: https://github.com/tufts-ml-courses/cs135-26s-assignments/tree/main/labs/data_abalone/README.md

In class (or later at home), work your way through this notebook. 

Each "part" has a structure like this:

* First, some prefilled content so you can learn how to do basic ML operations with sklearn.
* Second, some simple exercises to try to figure things out yourself


# Outline

* [Part 1: Linear Regression with 1-dim. features](#part1)
* * Real Task: Regression of 'rings' given 'length' using abalone dataset
* * Exercise 1a: Reproduce the `predict` method
* [Part 2: Linear Regression with 3-dim. features](#part2)
* * Real Task: Regression of 'rings' given 'length', 'height', 'weight'
* * Exercise 2a: Reproduce the `predict` method
* * Exercise 2b: Compare mean squared error on training set with model from part 1
* [Part 3: K-nearest neighbor regression](#part3)
* * Real Task: Regression of 'rings' given 'length'
* * Exercise 3a: How does training set error vary with number of neighbors?
* * Exercise 3b: Repeat using all available features (not just length)
* * Exercise 3c: Reproduce the `predict` method for k-NN


# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import sklearn

In [ ]:
# import plotting libraries
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', style='whitegrid', font_scale=1.25)

In [ ]:
os.getcwd()

In [ ]:
# If running in cloud on Google colab, be sure to acquire the required dataset
if os.getenv("COLAB_RELEASE_TAG") and not os.path.exists('data_abalone'):
    !git clone https://github.com/tufts-ml-courses/cs135-26s-assignments.git
    os.chdir(os.path.join('cs135-26s-assignments', 'labs'))
    
if not os.path.exists('data_abalone'):
    raise ValueError("Need to run this script within a valid checkout of the cs135 repo")

<a id="part1"></a>

# Part 1: Simple Linear Regression with 1-dim. features



### Goal: Predict 'rings' outcome given 'length' of abalone

We'll do this in a few easy steps below:

1) Load in 'x' training data, the 'length' of each abalone

2) Load in 'y' training data, the 'rings' count of each abalone

3) Train a linear regression model by calling 'fit'

4) Make predictions using this model by calling 'predict'

### Step 1: Load in 'x', the inputs for our regression task

In [ ]:
# Show the first 5 lines of this CSV:

with open('data_abalone/x_train.csv', 'r') as f:
    for line in f.readlines()[:5]:
        print(line.strip())

In [ ]:
# Load in the data into a Panda's dataframe called 'x_df'
x_df = pd.read_csv('data_abalone/x_train.csv')

In [ ]:
# Show first 3 rows
x_df.head(3)

In [ ]:
# Print summary stats
x_df.describe()

In [ ]:
# Show how to access column names
for col in x_df.columns:
    print(col)

In [ ]:
# Convert from pandas DataFrame to a numpy array with N rows and F columns
x_NF = x_df.values.copy()

print(type(x_NF))

In [ ]:
print(x_NF.shape)

In [ ]:
print(x_NF.dtype)

In [ ]:
# Convert only the 'length' column into a numpy array

xlength_N1 = x_df.loc[:, ['length_mm']].values.copy()

print(xlength_N1.shape)

OK, now we've got our input! Time for step 2....

### Step 2: Load in 'y', the output of our regression task

In [ ]:
y_df = pd.read_csv('data_abalone/y_train.csv')

In [ ]:
# Show first 6 rows
y_df.head(6)

In [ ]:
# Print summary stats
y_df.describe()

In [ ]:
y_N1 = y_df.loc[:, ['rings']].values.copy()

In [ ]:
y_N1.shape

### Step 3: Fit a linear regression model, using sklearn

In [ ]:
import sklearn.linear_model

In [ ]:
# Construct the regression "object"
# The variable 'lin_regr' here is an "instance" of the LinearRegression class

lin_regr = sklearn.linear_model.LinearRegression()

See the documentation:

<https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression>


Any constructed regressor in sklearn (like 'lin_regr') has two useful methods:
* 'fit' method
* 'predict' method (only works after 'fit' is called)

The 'fit' method will *TRAIN* the model

Practically, this method call will 

1) Estimate weight coefficients $w$ and bias/intercept $b$ to minimize mean squared error

2) Updates internal state of the object with these estimated values.

For linear regression specifically:

* an attribute called `coef_` holds the weights
* an attribute called `intercept_` holds the intercept value

In [ ]:
lin_regr.fit(xlength_N1, y_N1);

What changed after calling fit?

The *internal state* of the object was updated.

We can see the *internal state* via the "coef" attribute (the *slope* parameter) and the "intercept" attribute (aka *bias* parameter)

In [ ]:
lin_regr.coef_

In [ ]:
lin_regr.intercept_

How would you know what an object can do??

Pro tip: in an active Jupyter notebook, you can use *tab completion* to interactively inspect an object

Just type "lin_regr." and then press the <tab> key.
    
You should see a pop up menu that let's you see what methods and attributes are accessible, like this:
    
![Image of tab completion](https://www.cs.tufts.edu/comp/135/2020f/images/day02_jupyter_object_inspect_on_tab.png)

In [ ]:
# Try it here, add "." plus <tab>
lin_regr

### Step 4: Make predictions using this trained linear regression model

In [ ]:
# Prepare to evaluate predictions at several possible length values:
G = 7
x_grid_G = np.asarray([0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5])
print(x_grid_G)

In [ ]:
# Turn this into a G x 1 2-dim array, since sklearn *always* expects feature input to be 2-dim
x_grid_G1 = x_grid_G[:,np.newaxis]

Now, we call `predict`, which, when given 2-dim array with shape (G,F) of features, will return a 2-dim array of size (G, 1) of *predicted values*

In [ ]:
yhat_lin_G1 = lin_regr.predict(x_grid_G1)
print(yhat_lin_G1)
print(yhat_lin_G1.shape)

### Step 5: Plot the *predicted* values side by side with the observed training data

In [ ]:
plt.plot(xlength_N1, y_N1, 'k.', alpha=0.2)
plt.plot(x_grid_G, yhat_lin_G1, 'bs-', linewidth=3);
plt.xlabel('length (mm)'); plt.ylabel('ring count');
plt.title("Linear Regression using 'length' feature");

### Step 6: Calculate the mean squared error on the observed training data
 

In [ ]:
yhat_train_N1 = lin_regr.predict(xlength_N1)
sklearn.metrics.mean_squared_error(y_N1, yhat_train_N1)

# Part 2: Linear Regression with multiple features

### Goal: Predict 'rings' outcome given 'length', 'height', and 'weight' of abalone

How would we do if we wanted to predict 'rings' given not just length, but also height and weight?

### Prepare the data

We now want our features array to have shape: (n_examples, 3)

Since 3 would let us use length and height and weight

In [ ]:
for col in x_df.columns:
    print(col)

In [ ]:
# Grab relevant columns only, convert to a numeric array
x_N3 = x_df.loc[:, ['length_mm', 'height_mm', 'whole_weight_g']].values.copy()
print(type(x_N3))
print(x_N3.dtype)
print(x_N3.shape)

#### Construct the model as an sklearn regressor object

In [ ]:
lin_regr_3dim = sklearn.linear_model.LinearRegression()

#### Fit the model

In [ ]:
lin_regr_3dim.fit(x_N3, y_N1)

In [ ]:
# TODO: what are the learned weights?

In [ ]:
# TODO: what is the learned intercept?

# Group Activity: Feature Selection

With the person (or people) next to you, try to find the combination of 3 features that gives the best performance. Later we'll learn why this is not the best way to perform feature selection, but for now we'll consider one set of features as being better if it achieves lower mean squared error on the training data. 

Have you and your partner (or partners) each choose a different strategy for trying to find the best combination:

1. First find the best 1-feature model, then "lock-in" that feature and look for the second feature that best complements that first feature, then lock in 2 features and look for a third. 
2. Exhaustively trying all possible combinations of 3 features. 
3. Calculate the correlation between each feature and $y$ using `sklearn.feature_selection.r_regression` and choose the 3 most correlated features.
4. Vibe it out
5. Another strategy you think of.

Now with your group, consider the following questions:

1. Which strategy found the best performance? How much better was it than the other method(s)?
2. What is the worst-case time complexity (Big-O) of each strategy you tried, given in terms of the number of instances N and the number of features F?
3. Which strategy would you choose if you had 100 features? What about 1 million? 

<a id="part3"></a>

# Part 3: K-nearest Neighbor Regression with 1-dim features

### Goal: Try out sklearn's built-in nearest neighbor regressor



In [ ]:
import sklearn.neighbors

In [ ]:
k1nn_regr = sklearn.neighbors.KNeighborsRegressor(n_neighbors=1)

In [ ]:
# Fit the model
k1nn_regr.fit(xlength_N1, y_N1);

In [ ]:
# Prepare to evaluate predictions at several possible length values:
G = 21
x_grid_G = np.linspace(0, 1.5, G)
print(x_grid_G)

In [ ]:
x_grid_G1 = np.reshape(x_grid_G, (G, 1))

In [ ]:
# Make predictions on the same grid of x input values
yhat_k1nn_G1 = k1nn_regr.predict(x_grid_G1)

In [ ]:
plt.plot(xlength_N1, y_N1, 'k.', alpha=0.5)
plt.plot(x_grid_G, yhat_k1nn_G1, 'bs-', linewidth=3);
plt.xlabel('length (mm)'); plt.ylabel('ring count');
plt.title('K=1 Nearest Neighbor Regression');

#### Retry with 5 nearest neighbors

In [ ]:
k5nn_regr = sklearn.neighbors.KNeighborsRegressor(
    n_neighbors=5, algorithm='brute', metric='euclidean')

In [ ]:
# Fit the model
k5nn_regr.fit(xlength_N1, y_N1);

In [ ]:
# Make predictions on the same grid of x input values
yhat_k5nn_G1 = k5nn_regr.predict(x_grid_G1)

In [ ]:
plt.plot(xlength_N1, y_N1, 'k.', alpha=0.2)
plt.plot(x_grid_G1, yhat_k5nn_G1, 'bs-', linewidth=3);
plt.xlabel('length (mm)'); plt.ylabel('ring count');
plt.title('K=5 Nearest Neighbor Regression');

### Retry with 100 nearest neighbors

In [ ]:
k100nn_regr = sklearn.neighbors.KNeighborsRegressor(
    n_neighbors=100, algorithm='brute', metric='euclidean')

In [ ]:
# Fit the model
k100nn_regr.fit(xlength_N1, y_N1);

# Make predictions on the same grid of x input values
yhat_k100_G1 = k100nn_regr.predict(x_grid_G1)

In [ ]:
plt.plot(xlength_N1, y_N1, 'k.', alpha=0.2)
plt.plot(x_grid_G1, yhat_k100_G1, 'bs-', linewidth=3);
plt.xlabel('length (mm)'); plt.ylabel('ring count');
plt.title('K=100 Nearest Neighbor Regression');

# Exercises for Part 3

### Discussion Question 3(i): What happens when you call `fit` for a k-NN regressor? 

What (if anything) gets estimated or computed?
What (if anything) gets updated as an attribute of the regressor object?

### Discussion Question 3(ii): k-NN should have a piecewise constant predictions.... why do the plots above look different?

*Hint:* Try a finer grid

### Discussion Question 3(iii): Does the predicted function look smoother or more jagged as K increases? Why does this make sense given how this classifier works?

*Hint*: What should happen in the limit as K -> N?

### Exercise 3a (code): Make a plot of the mean squared error on the *training* set as a function of the number of neighbors

### Exercise 3b (code): Repeat part 3 using *all* available features (not just length). Does your performance improve?

# Challenge Yourself

### Exercise 3c (code): Can you write a function to reproduce what happens when you call `predict()` with kNN?